# Adding New Model Elements

This notebook walks through adding in-memory elements via PyMBE and tracking the additions to support updating model repositories after a session.

In [ ]:
import pymbe.api as pm
from importlib import resources as lib_resources

from uuid import uuid4

from pymbe.model import Element
from pymbe.model import MetaModel
from pymbe.model_modification import \
    build_from_classifier_pattern, new_element_ownership_pattern, build_from_feature_pattern, apply_covered_feature_pattern

from pymbe.metamodel import \
    get_more_general_types, derive_inherited_featurememberships

from pymbe.query.metamodel_navigator import get_effective_basic_name

## Open Library Data

The library data are organized around namespaces, where each namespace corresponds to a specific file in the library.

In [ ]:
library_model = None

with lib_resources.path("pymbe.static_data", "KernelLibrary.json") as lib_data:
    library_model = pm.Model.load_from_post_file(lib_data)

In [ ]:
library_model.ownedElement[0]._data

## New Package
The most basic element in a model for grouping things is the package. It holds a place within a modeling project to associate other model contents.

The new root-level Package also needs to be inside a Namespace and become an owned member via an OwningMembership.

In [ ]:
package_model_namespace_data = {
    'aliasIds': [],
    'isImpliedIncluded': False,
    '@type': "Namespace",
    '@id': str(uuid4()),
    'ownedRelationship': []
}

In [ ]:
package_model_data = {
    'name': "User Process Model",
    'isLibraryElement': False,
    'filterCondition': [],
    'ownedElement': [],
    'owner': {},
    '@type': "Package",
    '@id': str(uuid4()),
    'ownedRelationship': []
}

In [ ]:
empty_model = pm.Model(elements={})
empty_model

In [ ]:
new_ns = Element.new(data=package_model_namespace_data,model=empty_model)

In [ ]:
new_package = Element.new(data=package_model_data,model=empty_model)
new_package

In [ ]:
new_element_ownership_pattern(
    owner=new_ns, ele=new_package, model=empty_model, member_kind="OwningMembership"
)

### Connect New Model to Library

Reference the new model to the library so inheritance and redefinition can be connected to library elements.

In [ ]:
empty_model.reference_other_model(library_model)

In [ ]:
peform_ns = [library_model_ns
               for library_model_ns in library_model.ownedElement
               if library_model_ns.throughOwningMembership[0].declaredName == 'Performances'][0]

perform_eles = peform_ns.throughOwningMembership[0].throughOwningMembership

performance = None
    
for perform_ele in perform_eles:
    if perform_ele._metatype in ('Behavior'):
        if hasattr(perform_ele, "declaredName"):
            if perform_ele.declaredName == 'Performance':
                performance = perform_ele
                
performance

In [ ]:
performance._data

## New Performance

A new element to specialize the Performance library element is created.

In [ ]:
new_performance = build_from_classifier_pattern(
    owner=new_package,
    name="New Process",
    model=empty_model,
    metatype="Behavior",
    superclasses=[performance],
    specific_fields={"ownedRelationship": []}
)

In [ ]:
new_performance._derived

In [ ]:
new_package.throughOwningMembership

In [ ]:
new_process = new_package.throughOwningMembership[0]
new_process

In [ ]:
new_process._data

In [ ]:
new_process._derived

In [ ]:
get_more_general_types(new_process,0,20)

In [ ]:
new_process._data

In [ ]:
new_process.feature

In [ ]:
performance.ownedRelationship

## New Object

Create a basic Object that is connected back to the libarry

In [ ]:
object_ns = [library_model_ns
               for library_model_ns in library_model.ownedElement
               if library_model_ns.throughOwningMembership[0].declaredName == 'Objects'][0]

object_eles = object_ns.throughOwningMembership[0].throughOwningMembership

obj = None
    
for object_ele in object_eles:
    if object_ele._metatype in ('Structure'):
        if hasattr(object_ele, "declaredName"):
            if object_ele.declaredName == 'Object':
                obj = object_ele

In [ ]:
obj

## New Step
Add steps to our earlier new Performance.

In [ ]:
new_step_type_1 = build_from_classifier_pattern(
        owner=new_package,
        name="Sub-Process 1",
        model=empty_model,
        metatype="Behavior",
        superclasses=[performance],
        specific_fields={"ownedRelationship": []}
    )

new_step_type_2 = build_from_classifier_pattern(
    owner=new_package,
    name="Sub-Process 2",
    model=empty_model,
    metatype="Behavior",
    superclasses=[performance],
    specific_fields={"ownedRelationship": []}
)

new_step_1 = build_from_feature_pattern(
    owner=new_performance,
    name="Step 1",
    model=empty_model,
    specific_fields={},
    feature_type=new_step_type_1,
    direction="",
    metatype="Step",
    connector_end=False,
)

new_step_2 = build_from_feature_pattern(
    owner=new_performance,
    name="Step 2",
    model=empty_model,
    specific_fields={},
    feature_type=new_step_type_2,
    direction="",
    metatype="Step",
    connector_end=False,
)

In [ ]:
(new_step_1, new_step_2)

In [ ]:
new_performance.throughFeatureMembership

In [ ]:
self_feat = None
time_enclosed_feat = None

for new_perf_feature in new_performance.feature:
    if get_effective_basic_name(new_perf_feature) == "self":
        self_feat = new_perf_feature
    if get_effective_basic_name(new_perf_feature) == "timeEnclosedOccurrences":
        time_enclosed_feat = new_perf_feature

In [ ]:
new_performance_exec = build_from_classifier_pattern(
    owner=new_package,
    name="New Process as Executed",
    model=empty_model,
    metatype="Behavior",
    superclasses=[new_performance],
    specific_fields={"ownedRelationship": []}
)

step_1_exec = build_from_classifier_pattern(
    owner=new_package,
    name="Sub-Process 1 as Executed",
    model=empty_model,
    metatype="Behavior",
    superclasses=[new_step_type_1],
    specific_fields={"ownedRelationship": []}
)

step_2_exec = build_from_classifier_pattern(
    owner=new_package,
    name="Sub-Process 2 as Executed",
    model=empty_model,
    metatype="Behavior",
    superclasses=[new_step_type_2],
    specific_fields={"ownedRelationship": []}
)

In [ ]:
covered_self = apply_covered_feature_pattern(
    one_member_classifiers=[new_performance_exec],
    feature_to_cover=self_feat,
    type_to_apply_pattern_on=new_performance_exec,
    model=empty_model,
    new_types_owner=new_package,
    covering_classifier_prefix="Class to Cover ",
    covering_classifier_suffix="",
    redefining_feature_prefix="",
    redefining_feature_suffix="(Closed)",
)
covered_self

In [ ]:
covered_self._derived

In [ ]:
covered_self.reverseFeatureMembership

In [ ]:
covered_time_enclosed = apply_covered_feature_pattern(
    one_member_classifiers=[new_performance_exec, step_1_exec, step_2_exec],
    feature_to_cover=time_enclosed_feat,
    type_to_apply_pattern_on=new_performance_exec,
    model=empty_model,
    new_types_owner=new_package,
    covering_classifier_prefix="Class to Cover ",
    covering_classifier_suffix="",
    redefining_feature_prefix="",
    redefining_feature_suffix="(Closed)",
)

In [ ]:
covered_time_enclosed.throughFeatureTyping[0]._derived

In [ ]:
covered_time_enclosed.throughFeatureTyping[0].throughUnioning